# **Homework 2**

## First Iteration ##

In [ ]:
import tensorflow as tf

# Check if GPU is available
print('GPU is', 'available' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE')

In [ ]:
# Cell 1: Import Libraries
import numpy as np
import sys
import gymnasium as gym
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
)
from tensorflow.keras import optimizers, callbacks
from sklearn.metrics import classification_report, confusion_matrix
from gymnasium.wrappers import RecordVideo

In [ ]:
# Cell 2: Data Preparation
# Define the paths to your training and validation data
trainingset = 'train/'
validationset = 'test/'

batch_size = 64
target_size = (96, 96)  # Adjust based on your dataset

# Training data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    rotation_range=20,
    shear_range=0.1,
    fill_mode='nearest'
)

# Validation data should not be augmented
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    directory=trainingset,
    target_size=target_size,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    directory=validationset,
    target_size=target_size,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

# Gather dataset information
num_samples = train_generator.n
num_classes = train_generator.num_classes
input_shape = train_generator.image_shape

classnames = list(train_generator.class_indices.keys())
img_h, img_w, img_channels = input_shape
print(f"Image height = {img_h}, Image Width = {img_w}, Channels = {img_channels}")
print(f"Image input shape: {input_shape}")
print(f"Classes: {classnames}")
print(f"Loaded {num_samples} training samples from {num_classes} classes.")
print(f"Loaded {validation_generator.n} validation samples from {validation_generator.num_classes} classes.")

In [ ]:
# Cell X: Analyze Class Distribution
import matplotlib.pyplot as plt

# Count of samples per class
class_counts = train_generator.classes
class_names = list(train_generator.class_indices.keys())

plt.figure(figsize=(8, 6))
plt.bar(class_names, [np.sum(class_counts == i) for i in range(len(class_names))], color='skyblue')
plt.xlabel('Classes')
plt.ylabel('Number of Samples')
plt.title('Class Distribution in Training Set')
plt.show()

In [ ]:
# Cell 12: Create Separate Generators for Minority Classes
import os

# Define the target size and batch size
target_size = (96, 96)
batch_size = 64

# Initialize data augmentation for minority classes
minority_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    shear_range=0.2,
    fill_mode='nearest'
)

# Paths to minority class directories
minority_classes = ['0', '4']
train_dir = 'train/'

augmented_generators = []

for cls in minority_classes:
    class_dir = os.path.join(train_dir, cls)
    if not os.path.isdir(class_dir):
        print(f"Directory for class {cls} not found. Skipping augmentation.")
        continue
    generator = minority_datagen.flow_from_directory(
        directory=train_dir,
        target_size=target_size,
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        classes=[cls],  # Only target the minority class
        shuffle=True
    )
    augmented_generators.append(generator)

print(f"Number of augmented generators: {len(augmented_generators)}")

In [ ]:
# Cell Y: Compute Class Weights
from sklearn.utils import class_weight
import numpy as np

# Compute class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# Convert to a dictionary
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

In [ ]:
# Cell 13: Combine Original and Augmented Generators
import itertools

# Create a combined generator
train_combined = itertools.chain(
    train_generator,
    *augmented_generators
)

# Define a generator that yields data from the combined generator
def combined_generator(combined):
    for data in combined:
        yield data

# Reset the iterator
train_combined = combined_generator(train_combined)

In [ ]:
# Cell 5: Deeper CNN with Different Activation
def DeepCNN(input_shape, num_classes):
    model = Sequential(name="DeepCNN")

    # First Convolutional Block
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Second Convolutional Block
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Third Convolutional Block
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    # Flatten and Dense Layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model with RMSprop optimizer
    optimizer = optimizers.RMSprop(learning_rate=0.001)

    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    return model

# Instantiate and summarize the model
model_deep = DeepCNN(input_shape, num_classes)
model_deep.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.layers import LeakyReLU

def ImprovedDeepCNN(input_shape, num_classes):
    model = models.Sequential(name="ImprovedDeepCNN")
    
    # First Convolutional Block
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(LeakyReLU())
    model.add(layers.Conv2D(32, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(LeakyReLU())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))
    
    # Second Convolutional Block
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(LeakyReLU())
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(LeakyReLU())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))
    
    # Third Convolutional Block
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(LeakyReLU())
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(LeakyReLU())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.4))
    
    # Global Average Pooling and Dense Layers
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.001)))
    model.add(LeakyReLU())
    model.add(layers.Dropout(0.5))
    
    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Compile the model with Adam optimizer
    optimizer = optimizers.Adam(learning_rate=0.0001)
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    return model

# Instantiate and summarize the improved model
model_improved = ImprovedDeepCNN(input_shape, num_classes)
model_improved.summary()

In [ ]:
# Cell 7: Select Model
# Choose one of the models defined above
model = model_improved  # Replace with model_deep or model_improved

In [ ]:
# Cell 8: Define Callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,  # Increased patience for potentially longer training
    verbose=1,
    mode='auto',
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,  # Increased patience
    verbose=1,
    min_lr=1e-6
)

# Optionally, add ModelCheckpoint to save the best model
checkpoint = callbacks.ModelCheckpoint(
    'models/best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

In [ ]:
# Cell 9: Train the Model
# Calculate steps per epoch and validation steps
steps_per_epoch = int(np.ceil(train_generator.n / batch_size))
val_steps = int(np.ceil(validation_generator.n / batch_size))

try:
    history = model.fit(
        train_combined,
        epochs=100,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=val_steps,
        callbacks=[early_stopping, reduce_lr, checkpoint],
        class_weight=class_weights_dict,
        verbose=1
    )
except KeyboardInterrupt:
    print("Training interrupted.")

# Save the final model based if you chose deep or improved model
if model == model_deep:
    model.save('models/final_model_deep.keras')
else:
    model.save('models/final_model_improved.keras')

In [ ]:
from tensorflow.keras.layers import Input, Average
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model

model_deep = load_model('models/final_model_deep.keras')
model_improved = load_model('models/final_model_improved.keras')

input_shape_deep = model_deep.input_shape[1:]  # Exclude batch size
input_shape_improved = model_improved.input_shape[1:]

assert input_shape_deep == input_shape_improved, "Input shapes of both models must be the same."

# Create an input layer that matches the input shape
input_layer = Input(shape=input_shape_deep)

# Get predictions from both models
preds_deep = model_deep(input_layer)
preds_improved = model_improved(input_layer)

# Average the outputs
averaged_preds = Average()([preds_deep, preds_improved])

# Create the combined model
combined_model = Model(inputs=input_layer, outputs=averaged_preds)

# Compile the combined model
combined_model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

# Save the combined model
combined_model.save('models/combined_model.keras')

# Cell 11: Evaluate the Model
# Load the combined model
combined_model = load_model('models/combined_model.keras')

# Evaluate the model on the validation set
val_loss, val_acc = combined_model.evaluate(validation_generator, verbose=1)
print(f"Validation Accuracy: {val_acc:.4f}")

In [ ]:
# Cell 10: Evaluate the Model
# Load the best saved model
best_model = load_model('models/combined_model.keras')

# Evaluate on validation data
val_steps = int(np.ceil(validation_generator.n / batch_size))
loss, acc = best_model.evaluate(validation_generator, steps=val_steps, verbose=1)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {acc:.4f}')

In [ ]:
# Cell 11: Classification Report
preds = best_model.predict(validation_generator, steps=val_steps, verbose=0)
Ypred = np.argmax(preds, axis=1)
Ytest = validation_generator.classes  # Ensure shuffle=False in validation_generator

print(classification_report(Ytest, Ypred, target_names=classnames, digits=3))

In [ ]:
# Cell 12: Confusion Matrix
cm = confusion_matrix(Ytest, Ypred)

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=classnames, yticklabels=classnames, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Cell 13: Text-Based Confusion Matrix
conf = []
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        if i != j and cm[i][j] > 0:
            conf.append([i, j, cm[i][j]])

conf = np.array(conf)
conf = conf[np.argsort(-conf[:, 2])]  # Sort by descending error count

print(f'{"True":<16} {"Predicted":<16} {"Errors":<10} {"Error %":<10}')
print('-' * 60)
for k in conf:
    true_class = classnames[int(k[0])]
    pred_class = classnames[int(k[1])]
    errors = int(k[2])
    error_pct = (errors / validation_generator.n) * 100
    print(f'{true_class:<16} -> {pred_class:<16} {errors:<10} {error_pct:.2f}%')

In [ ]:
# Cell 9: Model Deployment with Gymnasium (Final Revised for Continuous Actions)
import numpy as np
from gymnasium.wrappers import RecordVideo

def play(env, model, predefined_actions):
    seed = 2000
    obs, _ = env.reset(seed=seed)

    # Drop initial frames with no action
    no_action = predefined_actions[0]  # [0.0, 0.0, 0.0]
    for _ in range(50):
        obs, _, _, _, _ = env.step(no_action)

    done = False
    while not done:
        # Preprocess the observation
        img = preprocess_observation(obs, target_size)
        p = model.predict(np.expand_dims(img, axis=0))  # Shape: (1, 5)
        predicted_class = np.argmax(p)  # Integer 0-4

        # Map the predicted class to a predefined action
        action = predefined_actions.get(predicted_class, predefined_actions[0])  # Array

        # Ensure the action is a float32 NumPy array
        action = action.astype(np.float32)

        # Step the environment with the action
        obs, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    env.close()

def preprocess_observation(obs, target_size):
    from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

    # Convert observation to PIL Image
    img = array_to_img(obs)
    # Resize image
    img = img.resize(target_size)
    # Convert to array and normalize
    img = img_to_array(img) / 255.0
    return img

# Define predefined actions (Continuous)
predefined_actions = {
    0: np.array([0.0, 0.0, 0.0], dtype=np.float32),  # No Action
    1: np.array([-1.0, 0.0, 0.0], dtype=np.float32), # Steer Left
    2: np.array([1.0, 0.0, 0.0], dtype=np.float32),  # Steer Right
    3: np.array([0.0, 1.0, 0.0], dtype=np.float32),  # Accelerate (Gas)
    4: np.array([0.0, 0.0, 1.0], dtype=np.float32),  # Brake
    # Add more actions as needed
}

# Initialize the environment without 'continuous' parameter
env_arguments = {
    'domain_randomize': False,
    'render_mode': 'rgb_array'
}

env_name = 'CarRacing-v3'
env = gym.make(env_name, **env_arguments)

# Wrap the environment to record videos
video_dir = 'video_recordings'  # Specify the directory to save video recordings
env = RecordVideo(env, video_dir)

print("Environment:", env_name)
print("Action space:", env.action_space)
print("Observation space:", env.observation_space)

# Play the game using the trained model
play(env, best_model, predefined_actions)
#play(env, best_model_regression, predefined_actions)

# (Optional Approach) Reinforcement Learning Model

In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from collections import deque
import random
import cv2
import os

# Suppress TensorFlow warnings for cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [11]:
# =========================
# 1. Define Predefined Actions
# =========================
predefined_actions = {
    0: np.array([0.0, 0.0, 0.0], dtype=np.float32),  # No Action
    1: np.array([-1.0, 0.0, 0.0], dtype=np.float32), # Steer Left
    2: np.array([1.0, 0.0, 0.0], dtype=np.float32),  # Steer Right
    3: np.array([0.0, 1.0, 0.0], dtype=np.float32),  # Accelerate (Gas)
    4: np.array([0.0, 0.0, 1.0], dtype=np.float32),  # Brake
}

In [12]:
# =========================
# 2. Define the Gym Environment
# =========================
class SimpleCarEnv(gym.Env):
    """
    Custom Gymnasium Environment for Car Driving with Basic Dynamics.
    """
    def __init__(self):
        super(SimpleCarEnv, self).__init__()
        # Define action space
        self.action_space = spaces.Discrete(len(predefined_actions))
        # Define observation space for preprocessed frames (RGB)
        self.observation_space = spaces.Box(low=0.0, high=1.0, shape=(96, 96, 3), dtype=np.float32)
        self.max_steps = 100
        self.current_step = 0
        self.position = np.array([48, 48], dtype=np.float32)  # Start at center (96/2, 96/2)
        self.speed = 0.0
        self.direction = 0.0  # Angle in degrees
        self.track_radius = 40  # Simple circular track
    
    def reset(self):
        """
        Reset the environment to an initial state and return the initial observation.
        """
        self.current_step = 0
        self.position = np.array([48, 48], dtype=np.float32)
        self.speed = 0.0
        self.direction = 0.0
        return self._get_observation()
    
    def step(self, action):
        """
        Apply the action to the environment and return the next observation, reward, done, and info.
        """
        self.current_step += 1
    
        # Apply action effects
        if action == 1:  # Steer Left
            self.direction -= 15  # degrees
        elif action == 2:  # Steer Right
            self.direction += 15  # degrees
        elif action == 3:  # Accelerate
            self.speed += 0.5
        elif action == 4:  # Brake
            self.speed -= 0.5
            self.speed = max(self.speed, 0.0)  # Prevent negative speed
    
        # Ensure direction is within [0, 360)
        self.direction %= 360
    
        # Update position based on speed and direction
        rad = np.deg2rad(self.direction)
        delta_x = self.speed * np.cos(rad)
        delta_y = self.speed * np.sin(rad)
        self.position += np.array([delta_x, delta_y], dtype=np.float32)
    
        # Calculate distance from center (assuming center is (48, 48))
        distance = np.linalg.norm(self.position - np.array([48, 48], dtype=np.float32))
    
        # Determine if off track
        done = False
        reward = 1.0  # Reward for staying on track
    
        if distance > self.track_radius:
            done = True
            reward = -100.0  # Off-track penalty
    
        if self.current_step >= self.max_steps:
            done = True
            reward = 100.0  # Success reward
    
        # Generate observation
        observation = self._get_observation()
    
        info = {
            'collision': False,  # Placeholder for collision detection
            'off_track': distance > self.track_radius,
            'speed': self.speed
        }
    
        return observation, reward, done, info
    
    def render(self, mode='human'):
        """
        Render the environment to the screen.
        """
        # Optional: Implement visualization using OpenCV or other libraries
        pass
    
    def _get_observation(self):
        """
        Generate and return the current observation.
        """
        # Create a blank RGB image
        img = np.zeros((240, 320, 3), dtype=np.float32)  # Using float32 for consistency
        
        # Draw the track (simple circular track)
        center_coordinates = (160, 120)  # Center of the image
        cv2.circle(img, center_coordinates, self.track_radius * 4, (0, 255, 0), 2)  # Green track
        
        # Draw the car's current position
        # Scale position from (0, 96) to image coordinates (0, 320) for x and (0, 96) to (0, 240) for y
        pos_x = int(self.position[0] * (320 / 96))
        pos_y = int(self.position[1] * (240 / 96))
        pos_x = np.clip(pos_x, 0, 319)
        pos_y = np.clip(pos_y, 0, 239)
        cv2.circle(img, (pos_x, pos_y), 5, (255, 0, 0), -1)  # Red car
        
        # Normalize pixel values to [0.0, 1.0]
        normalized = img / 255.0
        
        # Resize to (96, 96)
        resized = cv2.resize(normalized, (96, 96), interpolation=cv2.INTER_AREA)
        
        return resized  # Shape: (96, 96, 3)

In [13]:
# =========================
# 3. Preprocessing Functions
# =========================
def preprocess_observation(observation):
    """
    Preprocess the observation by performing any additional preprocessing steps if needed.
    Currently, the observation is already preprocessed within the environment.
    """
    # Placeholder: Additional preprocessing can be added here
    return observation

def compute_reward(info, done):
    """
    Compute the reward based on the current state.
    
    Parameters:
    - info: Dictionary containing additional information about the environment.
    - done: Boolean indicating if the episode has ended.
    
    Returns:
    - reward: Float value representing the reward.
    """
    if done:
        if info.get('collision', False):
            return -100.0  # Collision penalty
        elif info.get('off_track', False):
            return -100.0  # Off track penalty
        else:
            return 100.0  # Success reward
    else:
        reward = 1.0  # Reward for staying on track
        # Optionally, add more components (e.g., speed)
        reward += info.get('speed', 0.0) * 0.1
        return reward

In [14]:
# =========================
# 4. Frame Stacking Wrapper
# =========================
class FrameStackEnv(gym.Wrapper):
    """
    Gymnasium environment wrapper to stack consecutive frames.
    """
    def __init__(self, env, stack_size=4):
        super(FrameStackEnv, self).__init__(env)
        self.stack_size = stack_size
        self.frames = deque(maxlen=stack_size)
        # Update the observation space to reflect stacked frames
        low = np.repeat(env.observation_space.low, stack_size, axis=2)
        high = np.repeat(env.observation_space.high, stack_size, axis=2)
        self.observation_space = spaces.Box(low=0.0, high=1.0,
                                            shape=(env.observation_space.shape[0],
                                                   env.observation_space.shape[1],
                                                   env.observation_space.shape[2] * stack_size),
                                            dtype=np.float32)

    def reset(self):
        observation = self.env.reset()
        for _ in range(self.stack_size):
            self.frames.append(observation)
        return self._get_observation()

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.frames.append(observation)
        return self._get_observation(), reward, done, info

    def _get_observation(self):
        return np.concatenate(list(self.frames), axis=2)

In [15]:
# =========================
# 5. Replay Buffer
# =========================
class ReplayBuffer:
    def __init__(self, max_size=100000):
        self.buffer = deque(maxlen=max_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)

In [16]:
# =========================
# 6. DQN Model Architecture
# =========================
def build_dqn_model(input_shape, num_actions):
    """
    Build a Convolutional Neural Network for the DQN agent.
    """
    model = models.Sequential(name="DQN_Model")

    # Convolutional Layers
    model.add(layers.Conv2D(32, (8, 8), strides=4, activation='relu', input_shape=input_shape))
    model.add(layers.Conv2D(64, (4, 4), strides=2, activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), strides=1, activation='relu'))

    # Flatten and Dense Layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))

    # Output Layer
    model.add(layers.Dense(num_actions, activation='linear'))

    return model

In [17]:
# =========================
# 7. DQN Agent
# =========================
class DQNAgent:
    def __init__(self, input_shape, num_actions, learning_rate=1e-4, gamma=0.99,
                 epsilon=1.0, epsilon_min=0.1, epsilon_decay=1e-6, batch_size=32,
                 replay_buffer_size=100000, target_update_freq=1000):

        self.num_actions = num_actions
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.target_update_freq = target_update_freq
        self.learn_step_counter = 0

        # Main and Target Networks
        self.model = build_dqn_model(input_shape, num_actions)
        self.target_model = build_dqn_model(input_shape, num_actions)
        self.target_model.set_weights(self.model.get_weights())

        # Optimizer and Loss Function
        self.optimizer = optimizers.Adam(learning_rate=learning_rate)
        self.loss_function = tf.keras.losses.Huber()

        # Replay Buffer
        self.replay_buffer = ReplayBuffer(max_size=replay_buffer_size)

    def select_action(self, state):
        """
        Select an action using epsilon-greedy policy.
        
        Parameters:
        - state: Numpy array representing the current state.
        
        Returns:
        - action: Integer representing the chosen action.
        """
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.num_actions)
        state = np.expand_dims(state, axis=0)  # Add batch dimension
        q_values = self.model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def store_experience(self, state, action, reward, next_state, done):
        """
        Store experience tuple in the replay buffer.
        
        Parameters:
        - state: Current state.
        - action: Action taken.
        - reward: Reward received.
        - next_state: Next state after action.
        - done: Boolean indicating if the episode has ended.
        """
        self.replay_buffer.add((state, action, reward, next_state, done))

    def update_epsilon(self):
        """
        Decay epsilon after each training step to reduce exploration over time.
        """
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
        else:
            self.epsilon = self.epsilon_min

    def train_step(self):
        """
        Sample a batch from the replay buffer and perform a training step.
        """
        if len(self.replay_buffer) < self.batch_size:
            return

        batch = self.replay_buffer.sample(self.batch_size)
        states, actions, rewards, next_states, dones = map(np.array, zip(*batch))

        # Predict Q-values for next states using target network
        target_q = self.target_model.predict(next_states, verbose=0)
        max_target_q = np.max(target_q, axis=1)

        # Compute target values
        targets = rewards + (1 - dones) * self.gamma * max_target_q

        with tf.GradientTape() as tape:
            q_values = self.model(states, training=True)
            # Gather the Q-values for the taken actions
            action_masks = tf.one_hot(actions, self.num_actions)
            q_action = tf.reduce_sum(q_values * action_masks, axis=1)
            loss = self.loss_function(targets, q_action)

        # Backpropagation
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        # Update epsilon
        self.update_epsilon()

        # Update target network
        self.learn_step_counter += 1
        if self.learn_step_counter % self.target_update_freq == 0:
            self.target_model.set_weights(self.model.get_weights())

    def save_model(self, path):
        """
        Save the model's weights to the specified path.
        
        Parameters:
        - path: String representing the file path to save the weights.
        """
        self.model.save_weights(path)
        print(f"Model weights saved to {path}")

    def load_model(self, path):
        """
        Load the model's weights from the specified path.
        
        Parameters:
        - path: String representing the file path to load the weights from.
        """
        self.model.load_weights(path)
        self.target_model.set_weights(self.model.get_weights())
        print(f"Model weights loaded from {path}")

In [ ]:
# =========================
# 8. Training and Evaluation Functions
# =========================
def train_dqn(env, agent, num_episodes=1000, max_steps=1000, save_every=100, save_path='dqn_final_model.h5'):
    """
    Train the DQN agent in the given environment.

    Parameters:
    - env: The Gym environment.
    - agent: The DQN agent.
    - num_episodes: Total number of training episodes.
    - max_steps: Maximum steps per episode.
    - save_every: Frequency (in episodes) to save the model.
    - save_path: Path to save the final model.
    """
    for episode in range(1, num_episodes + 1):
        state = env.reset()
        total_reward = 0
        done = False

        for step in range(max_steps):
            action = agent.select_action(state)
            next_state, reward, done, info = env.step(action)
            total_reward += reward

            agent.store_experience(state, action, reward, next_state, done)
            agent.train_step()

            state = next_state

            if done:
                break

        print(f"Episode {episode}/{num_episodes} - Total Reward: {total_reward:.2f} - Epsilon: {agent.epsilon:.4f}")

        # Save the model at regular intervals
        if episode % save_every == 0:
            agent.save_model(f'models/reinforcement/dqn_model_episode_{episode}.weights.h5')

    # Save the final model
    agent.save_model(save_path)

def evaluate_agent(env, agent, num_episodes=10, max_steps=1000):
    """
    Evaluate the trained DQN agent without exploration.

    Parameters:
    - env: The Gym environment.
    - agent: The DQN agent.
    - num_episodes: Number of evaluation episodes.
    - max_steps: Maximum steps per episode.

    Returns:
    - avg_reward: Average reward over the evaluation episodes.
    """
    total_rewards = []
    original_epsilon = agent.epsilon
    agent.epsilon = 0.0  # Disable exploration

    for episode in range(1, num_episodes + 1):
        state = env.reset()
        total_reward = 0
        done = False

        for step in range(max_steps):
            action = agent.select_action(state)
            next_state, reward, done, info = env.step(action)
            total_reward += reward
            state = next_state
            if done:
                break

        total_rewards.append(total_reward)
        print(f"Evaluation Episode {episode}/{num_episodes} - Total Reward: {total_reward:.2f}")

    agent.epsilon = original_epsilon  # Restore original epsilon
    avg_reward = np.mean(total_rewards)
    print(f"Average Evaluation Reward over {num_episodes} episodes: {avg_reward:.2f}")
    return avg_reward

In [19]:
# =========================
# 9. Main Execution
# =========================


# Check for directory and create if it doesn't exist
if not os.path.exists('models/reinforcement'):
    os.makedirs('models/reinforcement')

# Initialize Environment and Wrap with FrameStack
env = SimpleCarEnv()
env = FrameStackEnv(env, stack_size=4)
# Define Input Shape and Number of Actions
input_shape = env.observation_space.shape  # Should be (96, 96, 3)
num_actions = env.action_space.n  # 5
print(f"Input Shape: {input_shape}")  # For debugging
# Initialize DQN Agent
agent = DQNAgent(
    input_shape=input_shape,
    num_actions=num_actions,
    learning_rate=1e-4,
    gamma=0.99,
    epsilon=1.0,
    epsilon_min=0.1,
    epsilon_decay=1e-6,
    batch_size=32,
    replay_buffer_size=100000,
    target_update_freq=1000
)
# Define Save Path (ensure this directory exists or change to a valid path)
save_path = 'models/reinforcement/dqn_final_model.weights.h5'
# Train the Agent
train_dqn(env, agent, num_episodes=100, max_steps=100, save_every=10, save_path=save_path)
# Evaluate the Agent
evaluate_agent(env, agent, num_episodes=10, max_steps=100)

Input Shape: (96, 96, 12)
Episode 1/100 - Total Reward: -40.00 - Epsilon: 1.0000
Episode 2/100 - Total Reward: 199.00 - Epsilon: 0.9999
Episode 3/100 - Total Reward: -67.00 - Epsilon: 0.9998
Episode 4/100 - Total Reward: -35.00 - Epsilon: 0.9998
Episode 5/100 - Total Reward: 199.00 - Epsilon: 0.9997
Episode 6/100 - Total Reward: 199.00 - Epsilon: 0.9996
Episode 7/100 - Total Reward: -18.00 - Epsilon: 0.9995
Episode 8/100 - Total Reward: -59.00 - Epsilon: 0.9994
Episode 9/100 - Total Reward: -34.00 - Epsilon: 0.9994
Episode 10/100 - Total Reward: -54.00 - Epsilon: 0.9993
Model weights saved to dqn_model_episode_10.weights.h5
Episode 11/100 - Total Reward: -47.00 - Epsilon: 0.9993
Episode 12/100 - Total Reward: -58.00 - Epsilon: 0.9992
Episode 13/100 - Total Reward: 199.00 - Epsilon: 0.9991
Episode 14/100 - Total Reward: -54.00 - Epsilon: 0.9991
Episode 15/100 - Total Reward: -79.00 - Epsilon: 0.9991
Episode 16/100 - Total Reward: -68.00 - Epsilon: 0.9990
Episode 17/100 - Total Reward: -

np.float64(199.0)

In [22]:
from tensorflow.keras.models import load_model
from gymnasium.wrappers import RecordVideo
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

target_size = (96, 96)

def play(env, agent, predefined_actions):
    seed = 2000
    obs, _ = env.reset(seed=seed)

    # Drop initial frames with no action
    no_action = predefined_actions[0]  # [0.0, 0.0, 0.0]
    for _ in range(50):
        obs, _, _, _, _ = env.step(no_action)

    done = False
    frames = deque(maxlen=4)  # Use deque to store the last 4 frames
    while not done:
        # Preprocess the observation
        img = preprocess_observation(obs, target_size)
        frames.append(img)
        if len(frames) < 4:
            continue  # Skip until we have 4 frames

        stacked_frames = np.concatenate(list(frames), axis=-1)
        stacked_frames = np.expand_dims(stacked_frames, axis=0)  # Add batch dimension
        p = agent.model.predict(stacked_frames, verbose=0)
        predicted_class = np.argmax(p)  # Integer 0-4

        # Map the predicted class to a predefined action
        action = predefined_actions.get(predicted_class, predefined_actions[0])  # Array

        # Ensure the action is a float32 NumPy array
        action = action.astype(np.float32)

        # Step the environment with the action
        obs, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    env.close()

def preprocess_observation(obs, target_size):
    # Convert observation to PIL Image
    img = array_to_img(obs)
    # Resize image
    img = img.resize(target_size)
    # Convert to array and normalize
    img = img_to_array(img) / 255.0
    return img

# Define predefined actions (Continuous)
predefined_actions = {
    0: np.array([0.0, 0.0, 0.0], dtype=np.float32),  # No Action
    1: np.array([-1.0, 0.0, 0.0], dtype=np.float32), # Steer Left
    2: np.array([1.0, 0.0, 0.0], dtype=np.float32),  # Steer Right
    3: np.array([0.0, 1.0, 0.0], dtype=np.float32),  # Accelerate (Gas)
    4: np.array([0.0, 0.0, 1.0], dtype=np.float32),  # Brake
    # Add more actions as needed
}

# Load the final model
agent.load_model(save_path)

# Run the agent in the environment and record a video

# Initialize the environment without 'continuous' parameter
env_arguments = {
    'domain_randomize': False,
    'render_mode': 'rgb_array'
}

env_name = 'CarRacing-v3'

env = gym.make(env_name, **env_arguments)

# Wrap the environment to record videos
video_dir = 'video_recordings'  # Specify the directory to save video recordings
env = RecordVideo(env, video_dir)

print("Environment:", env_name)
print("Action space:", env.action_space)
print("Observation space:", env.observation_space)

# Play the game using the trained model
play(env, agent, predefined_actions)

Model weights loaded from models/reinforcement/dqn_final_model.weights.h5
Environment: CarRacing-v3
Action space: Box([-1.  0.  0.], 1.0, (3,), float32)
Observation space: Box(0, 255, (96, 96, 3), uint8)
